In [3]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf

import os

#### Thay đổi hiển thị ####
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)
#### Change display ####

file_path = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/IoT23.csv", "C:\\Users\\hoang\\FileCSV_DACN_2025\\IoT23.csv"]
out_path =  ["/mnt/c/Users/hoang/FileCSV_DACN_2025/shuffle_IoT23.csv", "C:\\Users\\hoang\\FileCSV_DACN_2025\\shuffle_IoT23.csv"]

if os.name == 'nt':
    file_path = file_path[1]
    out_path = out_path[1]
else:
    file_path = file_path[0]
    out_path = out_path[0]

print(file_path)

dictTypes = {}
df = dk.read_csv(file_path)
for col in df.columns:
    if col.startswith('proto') == True:
        dictTypes[col] = 'int32'
    elif col.startswith('service_') == True:
        dictTypes[col] = 'int32'
    elif col == 'label':
        dictTypes[col]= 'int32'
    elif col.startswith('detailed-label'):
        dictTypes[col] = 'str'
    else:
        dictTypes[col]='float32'
del df

df =dk.read_csv(file_path, dtype = dictTypes, blocksize= "500MB")
# df = df.drop(columns=['Variance', 'Weight'])
print(df.npartitions)
print(df.dtypes)

df = df[df['detailed-label'].isin(['Okiru', 'PartOfAHorizontalPortScan', '0'])]

label_map = {
    '0': 0,
    'PartOfAHorizontalPortScan': 1,
    'Okiru': 2
}

df['detailed-label'] = df['detailed-label'].map(label_map).astype('int32')

print(df.head(50))

C:\Users\hoang\FileCSV_DACN_2025\IoT23.csv
86
id.orig_h                 float32
id.orig_p                 float32
id.resp_h                 float32
id.resp_p                 float32
duration                  float32
orig_bytes                float32
resp_bytes                float32
conn_state                float32
missed_bytes              float32
history                   float32
orig_pkts                 float32
orig_ip_bytes             float32
resp_pkts                 float32
resp_ip_bytes             float32
label                       int32
detailed-label    string[pyarrow]
proto_icmp                  int32
proto_tcp                   int32
proto_udp                   int32
service_dhcp                int32
service_dns                 int32
service_http                int32
service_irc                 int32
service_ssh                 int32
service_ssl                 int32
dtype: object


c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\dask_expr\_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('detailed-label', 'float64'))

  warnings.warn(meta_warning(meta))


    id.orig_h  id.orig_p  id.resp_h  id.resp_p    duration  orig_bytes  \
0    3.232236   0.513982   3.639551   0.423956   -0.003847   -0.013972   
1    3.232236   0.202751   4.026532   0.202751   12.061889   -0.013956   
2    3.232236   0.407997   3.639551   0.423956   -0.011165   -0.013972   
3    3.232236   0.855632   0.034618   0.873867   -0.012181   -0.013972   
4    3.232236   0.717052   3.232236   0.519432   -0.009679   -0.013972   
5    3.232236   0.202751   4.026532   0.202751   28.938551   -0.013940   
6    3.232236   0.540135   3.639551   0.423956    0.007909   -0.013972   
7    3.232236   0.701241   3.639551   0.423956   -0.004055   -0.013972   
8    3.232236   0.214855   3.639551   0.423956   -0.003828   -0.013972   
9    3.232236   0.370782   3.639551   0.423956   -0.011166   -0.013972   
10   3.232236   0.128592   0.034618   0.873867   -0.012181   -0.013972   
11   3.232236   0.247827   3.232236   0.519432   -0.009416   -0.013972   
12   3.232236   0.814629   3.639551   

In [4]:
from dask_ml.model_selection import train_test_split 
from sklearn.utils import shuffle

def shuffle_dask_dataframe(df, random_state=None):
    return (
        df.map_partitions(
            lambda part: part.assign(__shuffle_key=np.random.RandomState(random_state).random(len(part)))
        )
        .shuffle('__shuffle_key')
        .drop(columns='__shuffle_key')
    )
    
df = shuffle_dask_dataframe(df)

print(df.dtypes)
# print(df.npartitions)
print(df.head(50))

id.orig_h         float32
id.orig_p         float32
id.resp_h         float32
id.resp_p         float32
duration          float32
orig_bytes        float32
resp_bytes        float32
conn_state        float32
missed_bytes      float32
history           float32
orig_pkts         float32
orig_ip_bytes     float32
resp_pkts         float32
resp_ip_bytes     float32
label               int32
detailed-label      int32
proto_icmp          int32
proto_tcp           int32
proto_udp           int32
service_dhcp        int32
service_dns         int32
service_http        int32
service_irc         int32
service_ssh         int32
service_ssl         int32
dtype: object
86
      id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
87     3.232236   0.302162   2.671386   0.201239       0.0         0.0   
166    3.232236   0.981455   3.219872   0.201239       0.0         0.0   
190    3.232236   0.783453   3.435866   0.201239       0.0         0.0   
193    3.232236   0.357743   1.099598

In [3]:
print(df.head(200))

        id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
267      3.232236   0.992119   0.319840   0.201239       0.0         0.0   
1615     3.232236   0.892798   0.887647   0.201239       0.0         0.0   
2389     3.232236   0.252942   0.844416   0.201239       0.0         0.0   
2500     3.232236   0.116083   3.177330   0.201239       0.0         0.0   
2887     3.232236   0.139923   0.569805   0.201239       0.0         0.0   
2925     3.232236   0.870650   2.613936   0.201239       0.0         0.0   
3870     3.232236   0.531959   0.075272   0.201239       0.0         0.0   
3888     3.232236   0.366196   0.966937   0.201239       0.0         0.0   
5204     3.232236   0.407331   3.001555   0.201239       0.0         0.0   
6011     3.232236   0.226977   0.097909   0.201239       0.0         0.0   
6468     3.232236   0.107965   1.370842   0.201239       0.0         0.0   
9740     3.232236   0.935130   3.130743   0.201239       0.0         0.0   
10056    3.2

In [ ]:
header = True
for i in range(df.npartitions):
    df.get_partition(i).compute().to_csv(out_path, mode='a', header = header, index=False)
    header = False